![CC-BY-SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)
This notebook was created by [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc),
and is licensed under Creative Commons BY-SA

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Calculando todos os autovalores e autovetores

Pode ser que seja necessário conhecer todos os autovalores e autovetores de uma matriz.
Neste caso, uma solução (além de calcular todas as raízes do polinômio característico de $A$)
é criar uma série de transformações de $A$ que **preservam** os autovalores.
Da mesma forma como transformamos $A = PLU$ através de operações de linhas
(e poderíamos ter feito colunas também),
outras operações levam a matrizes com mesmos autovalores,
como é o caso de _similitudes_:
dizemos que $A$ é similar a $B$ se existe uma matriz $P$ tal que
$A = P^{-1} B P$.
Se $Av = \lambda v$, então
$$ P^{-1} B P v = \lambda v \Leftrightarrow B (Pv) = \lambda (Pv) $$
o que mostra que $Pv$ é um autovetor de $B$ com o mesmo autovalor.

Assim, buscamos uma seqüência de transformações que sejam similitudes.

## Fatoração QR e similitudes

Uma das maneiras mais curiosas de obter estas similitudes é a fatoração QR:
ela é bastante similar à fatoração LU: ainda há um bloco triangular superior, que agora chamamos $R$,
mas $Q$ é uma matriz mais especial: é uma matriz ortogonal,
ou seja, suas colunas (e linhas) são ortogonais entre si e de norma 1.
Isso (como já vimos na diagonalização) é muito útil para calcular inversas!

Assim, se temos $A = QR$, também é verdade que $Q^{-1}A = Q^T A = R$
e portanto a matriz $B = RQ$ é igual a $Q^{-1} A Q$, ou seja, $B$ é similar a $A$.
Também $A$ é similar a $B$, já que $A = Q B Q^{-1}$.

Vamos continuar: fatoramos $B$ novamente em $Q_2 R_2$, e permutamos,
obtendo $C = R_2 Q_2$.
Isso dá $B = Q_2 C Q_2^{-1}$, e portanto
$A = Q Q_2 C Q_2^{-1} Q^{-1}$.

Da mesma forma que iteramos a aplicação da matriz $A$ em um vetor $u$,
podemos ver a repetição desta "troca de lados" entre $Q$s e $R$s sucessivos como uma aplicação de $A$
aos vetores de $Q$ (que são ortogonais entre si, como uma base de autovetores de $A$ deve ser, se $A$ é simétrica).

Quando $A$ é simétrica, isso funciona exatamente assim:
os vetores dos produtos das $Q$s convergem para autovetores de $A$,
e a seqüência $A$, $B$, $C$, $\ldots$ converge para uma matriz diagnoal,
formada pelos autovalores de $A$ (que também são de $B$, $C$, etc.).

### Exercício

Implemente este algoritmo.
Inclua dois critérios de parada: um número máximo de iterações e uma tolerância.
Também retorne três valores: a matriz $Q$ limite, a matriz $R$ limite, e o número de iterações feitas.

In [2]:
def qrsequence(A, tol=1e-12, nmax=1000, debug=False):
    ### Resposta aqui


Verifique que o seu algoritmo calcula corretamente a decomposição de $A$:

In [3]:
A = np.array([[1,2,3], [2,4,8], [3,8,5]])
Qlim, Rlim, niter = qrsequence(A)
print(Qlim, Rlim, niter, sep='\n\n')
np.dot(Qlim,np.dot(Rlim,Qlim.T))

[[ 0.27348454  0.15400255 -0.94946797]
 [ 0.65158608  0.69645331  0.30064658]
 [ 0.70756045 -0.7008823   0.09012329]]

[[  1.35266810e+01  -8.92477224e-08   6.02359462e-16]
 [ -8.92477203e-08  -3.60862677e+00   2.12608013e-17]
 [  1.09156317e-29   2.78284590e-22   8.19458148e-02]]

13


array([[ 1.,  2.,  3.],
       [ 2.,  4.,  8.],
       [ 3.,  8.,  5.]])

Como você faria para verificar se os vetores coluna de $Q$ são realmente autovetores de $A$?

In [4]:
def verify_av(A,v):
    ### Resposta aqui


In [5]:
for v in Qlim.T:
    verify_av(A,v)

8.92477266759e-08
8.92477237375e-08
2.6049170547e-16


Agora, tente a seguinte matriz:

In [6]:
A = np.array([[1,2,3], [3,4,3], [3,6,5]])
Qlim, Rlim, niter = qrsequence(A, nmax=100)
np.dot(Qlim,np.dot(Rlim,Qlim.T)) - A

array([[  6.66133815e-16,  -8.88178420e-16,  -1.33226763e-15],
       [  1.33226763e-15,   0.00000000e+00,  -7.54951657e-15],
       [ -1.77635684e-15,  -2.66453526e-15,  -7.10542736e-15]])

In [7]:
niter

99

In [8]:
for i,v in enumerate(Qlim.T):
    verify_av(A,v)

8.48431244129e-15
2.67041711453
1.20010094753


Porque isso ocorre?

In [9]:
print(Qlim, end='\n\n')
print(Rlim)

[[ 0.35627459 -0.56327552  0.74551265]
 [ 0.52481075 -0.5394954  -0.65842114]
 [ 0.77307315  0.62583177  0.10340454]]

[[  1.04557428e+001  -2.33994248e+000  -1.06399478e+000]
 [  3.17777415e-107  -1.94544318e-001  -5.55119269e-001]
 [  2.26485138e-107   1.28677766e+000  -2.61198506e-001]]


# Fatoração / Decomposição SVD

SVD (_Singular Value Decomposition_) é uma outra fatoração de matrizes,
mais geral do que a diagonalização, e que serve também para matrizes retangulares.
Essencialmente, ela consiste em encontrar **duas** bases
(em vez de apenas uma que é o caso de autovetores da diagonalização)
que sejam adaptadas à matriz $A$.
Assim, queremos escrever
$$ A = U \Sigma V^{-1} , $$
onde $U$ e $V$ são ortogonais (correspondendo a mudanças de base "por rotação")
e $\Sigma$ é uma matriz diagonal (correspondendo a multiplicações entre as bases).

Neste caso, a idéia é usar uma matriz simétrica
(para a qual sabemos existir a diagonalização e bases "apropriadas")
associada a $A$: $A^T A$.
Parece apelação, mas ao supor a decomposição $A = U \Sigma V^{-1}$,
com matrizes ortogonais $U$ e $V$, temos
$$ A^T A = {V^{-1}}^T \Sigma^T U^T U \Sigma V^{-1} = V \Sigma^T \Sigma V^{-1} $$
que é **a diagonalização** de $A^T A$, se tomarmos $\Sigma^T \Sigma = D$.

Assim, podemos obter uma base de "vetores especiais" a partir de $V$,
e em seguida tomamos uma matriz diagonal $\Sigma$ tal que $\Sigma^T \Sigma = D$;
o mais simples é usar a raiz quadrada dos elementos diagonais de $D$
(que são positivos porque $A^T A$ é simétrica!).
Enfim, para obter $U$, basta resolver o sistema $A = U \Sigma V^{-1}$,
que é linear em $U$.

### Exercício

Escreva um código para a decomposição SVD.
Para diagonalizar $A^T A$, você pode usar o cálculo de autovalores e autovetores da iteração QR.

In [10]:
def svd(A, tol=1e-12, nmax=1000, debug=False):
    ### Resposta aqui

    return U, Sigma, V

In [11]:
u,s,v = svd(A)
print(u,s,v, sep='\n\n')

[[-0.3332243  -0.78664991  0.51975336]
 [-0.5354972   0.61162192  0.58237558]
 [-0.77601824 -0.08426472 -0.6250561 ]]

[ 10.77055427   1.28977222   0.57588917]

[[-0.39624429  0.61671489  0.68018616]
 [-0.69305132  0.28501119 -0.66215443]
 [-0.60222116 -0.73377883  0.31448101]]


Como verificar que esta é de fato a decomposição SVD de $A$?

In [12]:
### Resposta aqui


array([[ -1.11022302e-15,  -4.44089210e-16,  -8.88178420e-16],
       [ -8.88178420e-16,  -1.77635684e-15,  -1.33226763e-15],
       [ -1.77635684e-15,  -2.66453526e-15,  -8.88178420e-16]])